In [1]:
import logging
import pandas as pd
import library_pipeline
from library_pipeline import ExtractDrtTrips as edt
# import PlansManipulation
from library_pipeline import SpacialOperations as sops
from library_pipeline import ExtractHubs
# import pyinterpolate
from library_pipeline import Legs2df
from library_pipeline import InterpolateTrips
from library_pipeline import GTFSpreprocessing
from library_pipeline import createGTFS
from library_pipeline import importCells
from library_pipeline import joinAndexportGTFS

In [85]:
# Constants
legs_input_path = 'Paris/output_legs.csv' # MATSim output legs csv file
studyarea_input_path = 'Paris/shp/cps.shp' # shapefile .shp for the DRT study area in this case the study area is Paris-saclay
popArea_input_path = 'Paris/shp/ParisPoP.shp' # the population of paris 
gtfs_input_dir = 'Paris/gtfs/' # path to where gtfs.zip is stored

# EPSG code !!ONLY NUMBER!! EPSG:3035 --> 3035
crs_import = 2154 #2154 # crs of the raw imported data
crs_working = 2154 #2154 # crs used for the processing (can be same as crs_import)

# Toggle for preprocessing steps
# GTFS preprocessing to get hexagonal grid
#      --> needed once, can be skipped if GTFScells.csv file exists
toggle_GTFSprep = False
GTFScells_path = 'Paris/output/GTFScells_basecase_ile_de_france2.csv'

#the path to where the new gtfs files will be stored
gtfs_outputPath ="Paris/output/gtfs"


# Interpolation will only be done in this time window
start_time = '07:00:00'
end_time = '19:00:00'
# Length of each analysis interval (the estimation of travel time and wait time will be done for each deltaT interval)
deltaT = 1 # in hours (h), default 1h

In [8]:
# Setup of the logging module
logging.basicConfig(filename='Paris/output/Plans2GTFS.log',
                    level=logging.DEBUG)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
logging.getLogger('').addHandler(console)

In [86]:
#Read the csv file of trips and extract drt trips only
df = Legs2df.run(legs_input_path)

C:\Users\mourahou\Documents\Severin's paper\library_pipeline\Legs2df.py:8: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=';', header=0)


In [87]:
df

,agentID,trip_id,<route>_distance,start_link,start_x,start_y,end_link,end_x,end_y,vehicle_id,<leg>_trav_time,wait_time,<leg>_dep_time
272,10004937,10004937_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_93,443,277,28822
1215,1001879,1001879_1,2394,30347,645334.224928,6.847593e+06,249276,645178.745516,6.848194e+06,vehicle_drt_262,417,61,27593
3190,10035536,10035536_2,780,373905,634006.318125,6.844592e+06,418142,633676.048264,6.844983e+06,vehicle_drt_139,136,161,53076
3579,10038603,10038603_3,734,28293,642930.633995,6.849632e+06,523336,643382.681350,6.849181e+06,vehicle_drt_22,121,96,41275
4260,10043934,10043934_1,1141,52986,644475.791845,6.846566e+06,492973,644505.725852,6.845947e+06,vehicle_drt_259,208,16,28345
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173637,998386,998386_1,10051,363799,642117.061658,6.845355e+06,246639,642372.295495,6.843694e+06,vehicle_drt_167,1557,198,30118
4174587,9998790,9998790_2,6887,255857,638650.536115,6.848612e+06,537171,640676.987241,6.845386e+06,vehicle_drt_125,682,178,56718
4174600,999887,999887_1,2818,13460,639578.460420,6.844363e+06,553637,641211.307923,6.842971e+06,vehicle_drt_39,483,551,30480
4174612,9998939,9998939_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_101,384,457,31222


In [88]:
#convert the coordinates to the choosed crs_working
edt.convertCRS(df,crs_import, crs_working,'start_x', 'start_y','end_x', 'end_y')

In [89]:
edt.constructWktPoint(df, 'start_x', 'start_y', 'origin')
edt.constructWktPoint(df, 'end_x', 'end_y', 'destination')

In [90]:
df = sops.createGeoDataFrame(df['origin'], 'EPSG:' + str(crs_working), df)

In [91]:
df = df.to_crs(crs=crs_working)

In [92]:
DfHubs = ExtractHubs.run(df)

In [93]:
DfHubs

,position,Count,name
0,POINT (633676.0482636443 6844983.164553668),198,Hub_0
1,POINT (636454.5534391978 6844596.322888183),730,Hub_1
2,POINT (637569.4661344511 6844167.124794338),185,Hub_2
3,POINT (638478.3893558099 6844294.495396683),421,Hub_3
4,POINT (639578.4604200339 6844363.259593032),829,Hub_4
5,POINT (640676.9872406526 6845385.660893883),427,Hub_5
6,POINT (642117.061658004 6845354.874643082),257,Hub_6
7,POINT (643382.6813501306 6849180.759473929),763,Hub_7
8,POINT (643881.6021442156 6845710.885985972),603,Hub_8
9,POINT (644475.7918446767 6846565.807303847),1243,Hub_9


In [94]:
ExtractHubs.assignHub2Trip(DfHubs, df)

In [133]:
DfHubs.to_csv("Paris/output/hubs.csv",sep=';')

In [50]:
if toggle_GTFSprep: GTFSpreprocessing.run(GTFScells_path)

In [95]:
#Read the polygones file (the study area is devided into polygones )
CellsDF = importCells.importcells_run(GTFScells_path, crs_working)
CellsDF

,geometry,Centroids,Centroid_X,Centroid_Y
0,"POLYGON ((566321.000 6927138.000, 566061.000 6...",POINT (565821.4538022822 6927149.39923337),565821.453802,6.927149e+06
1,"POLYGON ((566301.000 6926271.000, 566041.000 6...",POINT (565801.164492585 6926283.19106447),565801.164493,6.926283e+06
2,"POLYGON ((566281.000 6925405.000, 566021.000 6...",POINT (565780.875353718 6925416.990188788),565780.875354,6.925417e+06
3,"POLYGON ((566261.000 6924539.000, 566001.000 6...",POINT (565760.5863852837 6924550.796589374),565760.586385,6.924551e+06
4,"POLYGON ((566241.000 6923673.000, 565980.000 6...",POINT (565740.2975868857 6923684.610249288),565740.297587,6.923685e+06
...,...,...,...,...
44338,"POLYGON ((742264.000 6765977.000, 742010.000 6...",POINT (741750.08302322 6765973.095236102),741750.083023,6.765973e+06
44339,"POLYGON ((742270.000 6765111.000, 742016.000 6...",POINT (741756.2173419723 6765107.73353937),741756.217342,6.765108e+06
44340,"POLYGON ((742277.000 6764246.000, 742023.000 6...",POINT (741762.3516308034 6764242.376063571),741762.351631,6.764242e+06
44341,"POLYGON ((742283.000 6763381.000, 742029.000 6...",POINT (741768.4858898298 6763377.022792322),741768.485890,6.763377e+06


In [96]:

CellsDF = sops.removeExcessGrid(CellsDF, studyarea_input_path, crs_working, popArea_input_path, False)
CellsDF

,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea
17338,0,"POLYGON ((633659.000 6844974.000, 633401.000 6...",POINT (633152.008002235 6844979.32807629),633152.008002,6.844979e+06,True
17339,1,"POLYGON ((633649.000 6844108.000, 633391.000 6...",POINT (633142.0477094634 6844113.548845413),633142.047709,6.844114e+06,True
17523,2,"POLYGON ((634464.000 6848861.000, 634206.000 6...",POINT (633957.1260910208 6848866.708908299),633957.126091,6.848867e+06,True
17524,3,"POLYGON ((634454.000 6847995.000, 634196.000 6...",POINT (633947.278862393 6848000.902416679),633947.278862,6.848001e+06,True
17525,4,"POLYGON ((634444.000 6847129.000, 634186.000 6...",POINT (633937.4316997194 6847135.101724004),633937.431700,6.847135e+06,True
...,...,...,...,...,...,...
21890,342,"POLYGON ((651890.000 6841331.000, 651633.000 6...",POINT (651382.6819698152 6841334.763588596),651382.681970,6.841335e+06,True
22070,343,"POLYGON ((652718.000 6849550.000, 652462.000 6...",POINT (652211.6651272465 6849553.684946085),652211.665127,6.849554e+06,True
22071,344,"POLYGON ((652711.000 6848684.000, 652454.000 6...",POINT (652204.5386051976 6848687.84596957),652204.538605,6.848688e+06,True
22072,345,"POLYGON ((652704.000 6847818.000, 652447.000 6...",POINT (652197.412131017 6847822.012808847),652197.412131,6.847822e+06,True


In [121]:
CellsDF.to_csv('Paris/output/CellsDF_basecase_studyArea.csv',sep=';')

In [97]:

CellsDF = sops.countTripsPerCell(CellsDF, df)
CellsDF

,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea,origin_count
0,0,"POLYGON ((633659.000 6844974.000, 633401.000 6...",POINT (633152.008002235 6844979.32807629),633152.008002,6.844979e+06,True,1.0
1,1,"POLYGON ((633649.000 6844108.000, 633391.000 6...",POINT (633142.0477094634 6844113.548845413),633142.047709,6.844114e+06,True,0.0
2,2,"POLYGON ((634464.000 6848861.000, 634206.000 6...",POINT (633957.1260910208 6848866.708908299),633957.126091,6.848867e+06,True,0.0
3,3,"POLYGON ((634454.000 6847995.000, 634196.000 6...",POINT (633947.278862393 6848000.902416679),633947.278862,6.848001e+06,True,0.0
4,4,"POLYGON ((634444.000 6847129.000, 634186.000 6...",POINT (633937.4316997194 6847135.101724004),633937.431700,6.847135e+06,True,0.0
...,...,...,...,...,...,...,...
342,342,"POLYGON ((651890.000 6841331.000, 651633.000 6...",POINT (651382.6819698152 6841334.763588596),651382.681970,6.841335e+06,True,0.0
343,343,"POLYGON ((652718.000 6849550.000, 652462.000 6...",POINT (652211.6651272465 6849553.684946085),652211.665127,6.849554e+06,True,0.0
344,344,"POLYGON ((652711.000 6848684.000, 652454.000 6...",POINT (652204.5386051976 6848687.84596957),652204.538605,6.848688e+06,True,0.0
345,345,"POLYGON ((652704.000 6847818.000, 652447.000 6...",POINT (652197.412131017 6847822.012808847),652197.412131,6.847822e+06,True,0.0


In [98]:
sops.addCellId2Trip(CellsDF, df)
df

,agentID,trip_id,<route>_distance,start_link,start_x,start_y,end_link,end_x,end_y,vehicle_id,<leg>_trav_time,wait_time,<leg>_dep_time,origin,destination,geometry,tripType,Hub,originCell
272,10004937,10004937_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_93,443,277,28822,POINT (640676.9872406526 6845385.660893883),POINT (638650.5361152911 6848611.9949821625),POINT (640676.987 6845385.661),egress,Hub_5,116
1215,1001879,1001879_1,2394,30347,645334.224928,6.847593e+06,249276,645178.745516,6.848194e+06,vehicle_drt_262,417,61,27593,POINT (645334.2249275153 6847592.524996355),POINT (645178.7455157 6848193.557723006),POINT (645334.225 6847592.525),egress,Hub_10,223
3190,10035536,10035536_2,780,373905,634006.318125,6.844592e+06,418142,633676.048264,6.844983e+06,vehicle_drt_139,136,161,53076,POINT (634006.3181254587 6844591.817313871),POINT (633676.0482636443 6844983.164553668),POINT (634006.318 6844591.817),access,Hub_0,6
3579,10038603,10038603_3,734,28293,642930.633995,6.849632e+06,523336,643382.681350,6.849181e+06,vehicle_drt_22,121,96,41275,POINT (642930.6339947096 6849631.82615489),POINT (643382.6813501306 6849180.759473929),POINT (642930.634 6849631.826),access,Hub_7,161
4260,10043934,10043934_1,1141,52986,644475.791845,6.846566e+06,492973,644505.725852,6.845947e+06,vehicle_drt_259,208,16,28345,POINT (644475.7918446767 6846565.807303847),POINT (644505.725852222 6845946.915065141),POINT (644475.792 6846565.807),egress,Hub_9,204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173637,998386,998386_1,10051,363799,642117.061658,6.845355e+06,246639,642372.295495,6.843694e+06,vehicle_drt_167,1557,198,30118,POINT (642117.061658004 6845354.874643082),POINT (642372.2954946852 6843693.965287579),POINT (642117.062 6845354.875),egress,Hub_6,148
4174587,9998790,9998790_2,6887,255857,638650.536115,6.848612e+06,537171,640676.987241,6.845386e+06,vehicle_drt_125,682,178,56718,POINT (638650.5361152911 6848611.9949821625),POINT (640676.9872406526 6845385.660893883),POINT (638650.536 6848611.995),access,Hub_5,68
4174600,999887,999887_1,2818,13460,639578.460420,6.844363e+06,553637,641211.307923,6.842971e+06,vehicle_drt_39,483,551,30480,POINT (639578.4604200339 6844363.259593032),POINT (641211.3079227756 6842971.122922318),POINT (639578.460 6844363.260),egress,Hub_4,103
4174612,9998939,9998939_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_101,384,457,31222,POINT (640676.9872406526 6845385.660893883),POINT (638650.5361152911 6848611.9949821625),POINT (640676.987 6845385.661),egress,Hub_5,116


In [105]:
# for each hub and for each deltaT interval the estimation of travel time and wait time at unsampled areas are computed
#in this notebook LineaRegression is used to make estimation, you can change it to your suited model
CellsDF =runAllIntervals(df, CellsDF, deltaT, [start_time, end_time], DfHubs)

Interpolation for time interval:   0%|                                                          | 0/12 [00:00<?, ?it/s]C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-999', inplace=True)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future er


Interval 1
Interpolation for Time interval 7:0:00 to 8:0:00
Hub_5
Interpolating Access Trips
Found 40 trips to interpolate
Interpolating Egress Trips
Found 55 trips to interpolate
Hub_10
Interpolating Access Trips
Found 89 trips to interpolate
Interpolating Egress Trips
Found 46 trips to interpolate
Hub_0
Interpolating Access Trips
Found 19 trips to interpolate
Interpolating Egress Trips
Found 11 trips to interpolate
Hub_7
Interpolating Access Trips
Found 38 trips to interpolate
Interpolating Egress Trips
Found 21 trips to interpolate
Hub_9
Interpolating Access Trips
Found 103 trips to interpolate


C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-999', inplace=True)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-999', inplace=True)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with

Interpolating Egress Trips
Found 66 trips to interpolate
Hub_4
Interpolating Access Trips
Found 82 trips to interpolate
Interpolating Egress Trips
Found 149 trips to interpolate
Hub_6
Interpolating Access Trips
Found 45 trips to interpolate
Interpolating Egress Trips
Found 33 trips to interpolate
Hub_8
Interpolating Access Trips
Found 60 trips to interpolate
Interpolating Egress Trips
Found 50 trips to interpolate
Hub_13
Interpolating Access Trips
Found 21 trips to interpolate
Interpolating Egress Trips
Found 23 trips to interpolate
Hub_3
Interpolating Access Trips
Found 35 trips to interpolate
Interpolating Egress Trips
Found 42 trips to interpolate
Hub_12
Interpolating Access Trips
Found 71 trips to interpolate
Interpolating Egress Trips
Found 50 trips to interpolate
Hub_11
Interpolating Access Trips
Found 28 trips to interpolate
Interpolating Egress Trips
Found 16 trips to interpolate
Hub_1
Interpolating Access Trips
Found 109 trips to interpolate
Interpolating Egress Trips
Found 38

C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-999', inplace=True)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  

...


Interval 12
Interpolation for Time interval 18:0:00 to 19:0:00
Hub_5
Interpolating Access Trips
Found 27 trips to interpolate
Interpolating Egress Trips
Found 49 trips to interpolate
Hub_10
Interpolating Access Trips
Found 43 trips to interpolate
Interpolating Egress Trips
Found 120 trips to interpolate
Hub_0
Interpolating Access Trips
Found 7 trips to interpolate
Interpolating Egress Trips
Found 14 trips to interpolate
Hub_7
Interpolating Access Trips
Found 39 trips to interpolate
Interpolating Egress Trips
Found 74 trips to interpolate
Hub_9
Interpolating Access Trips
Found 47 trips to interpolate
Interpolating Egress Trips
Found 146 trips to interpolate
Hub_4
Interpolating Access Trips
Found 33 trips to interpolate


C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-999', inplace=True)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  

Interpolating Egress Trips
Found 52 trips to interpolate
Hub_6
Interpolating Access Trips
Found 7 trips to interpolate
Interpolating Egress Trips
Found 17 trips to interpolate
Hub_8
Interpolating Access Trips
Found 40 trips to interpolate
Interpolating Egress Trips
Found 45 trips to interpolate
Hub_13
Interpolating Access Trips
Found 3 trips to interpolate
Interpolating Egress Trips
Found 59 trips to interpolate
Hub_3
Interpolating Access Trips
Found 13 trips to interpolate
Interpolating Egress Trips
Found 32 trips to interpolate
Hub_12
Interpolating Access Trips
Found 16 trips to interpolate
Interpolating Egress Trips
Found 103 trips to interpolate


C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-999', inplace=True)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mourahou\AppData\Local\Temp\ipykernel_23336\2978660176.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-999' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-999', inplace=True)
C:\Users\mourahou\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with

Hub_11
Interpolating Access Trips
Found 22 trips to interpolate
Interpolating Egress Trips
Found 101 trips to interpolate
Hub_1
Interpolating Access Trips
Found 19 trips to interpolate
Interpolating Egress Trips
Found 59 trips to interpolate
Hub_14
Interpolating Access Trips
Found 17 trips to interpolate
Interpolating Egress Trips
Found 43 trips to interpolate
Hub_15
Interpolating Access Trips
Found 0 trips to interpolate
Interpolating Egress Trips
Found 15 trips to interpolate
Hub_2
Interpolating Access Trips
Found 5 trips to interpolate
Interpolating Egress Trips
Found 16 trips to interpolate


In [ ]:
#create the a dataframe of new gtfs files
drtGTFS = createGTFS.run(CellsDF, DfHubs, deltaT, [start_time, end_time],crs_working)

adding all stops to gtfs: stops.txt: 100%|█████████████████████████████████████████| 363/363 [00:00<00:00, 2112.14it/s]


Creating stops in cells
Creating stops at hubs


adding all waiting and travel times to gtfs: stop_times.txt:   0%|                             | 0/363 [00:00<?, ?it/s]C:\Users\mourahou\Documents\Severin's paper\library_pipeline\createGTFS.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cellsCLEAN = pd.concat([cellsCLEAN,cell], ignore_index=True)
adding all waiting and travel times to gtfs: stop_times.txt:  81%|█████████████▊   | 294/363 [2:33:33<41:50, 36.39s/it]

In [104]:
exportGTFS(outputPath, drtGTFS)